<a href="https://colab.research.google.com/github/skhabiri/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/skhabiri_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://github.com/skhabiri/PredictiveModeling-LinearModels-u2s1/tree/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

In [3]:
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [4]:
df["SALE PRICE"]

0              $   - 0
1              $   - 0
2              $   - 0
3              $   - 0
4              $   - 0
             ...      
23035      $   510,000
23036      $   355,000
23037      $   375,000
23038    $   1,100,000
23039      $   569,202
Name: SALE PRICE, Length: 23040, dtype: object

In [5]:
# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [6]:
df["SALE_PRICE"][50:]

50             0
51             0
52             0
53             0
54             0
          ...   
23035     510000
23036     355000
23037     375000
23038    1100000
23039     569202
Name: SALE_PRICE, Length: 22990, dtype: int64

In [7]:
df['BOROUGH']

0        1
1        1
2        1
3        1
4        1
        ..
23035    4
23036    4
23037    4
23038    4
23039    4
Name: BOROUGH, Length: 23040, dtype: int64

In [21]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [22]:
df['BOROUGH']

44       3
61       4
78       2
108      3
111      3
        ..
23029    4
23031    4
23032    4
23033    4
23035    4
Name: BOROUGH, Length: 3151, dtype: object

In [8]:
df['NEIGHBORHOOD']

0                        CHELSEA
1                        FASHION
2                        FASHION
3         GREENWICH VILLAGE-WEST
4        UPPER EAST SIDE (59-79)
                  ...           
23035                 ST. ALBANS
23036                  SUNNYSIDE
23037                  SUNNYSIDE
23038                  WOODHAVEN
23039                   WOODSIDE
Name: NEIGHBORHOOD, Length: 23040, dtype: object

In [9]:
df['NEIGHBORHOOD'].value_counts()

FLUSHING-NORTH               685
UPPER EAST SIDE (59-79)      465
UPPER EAST SIDE (79-96)      410
BEDFORD STUYVESANT           408
BOROUGH PARK                 376
                            ... 
BRONX PARK                     1
DONGAN HILLS-OLD TOWN          1
FRESH KILLS                    1
RICHMONDTOWN-LIGHTHS HILL      1
CITY ISLAND-PELHAM STRIP       1
Name: NEIGHBORHOOD, Length: 252, dtype: int64

In [10]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [13]:
df['NEIGHBORHOOD'].isin(top10)

0        False
1        False
2        False
3        False
4         True
         ...  
23035    False
23036    False
23037    False
23038    False
23039    False
Name: NEIGHBORHOOD, Length: 23040, dtype: bool

In [14]:
df.shape

(23040, 21)

In [15]:
df["SALE_PRICE"]

0              0
1              0
2              0
3              0
4              0
          ...   
23035     510000
23036     355000
23037     375000
23038    1100000
23039     569202
Name: SALE_PRICE, Length: 23040, dtype: int64

In [16]:
# keeping samples from a certain range of SALE_PRICE and a specific BULDING_CLASS_CATEGORY.
df = df[(df["BUILDING_CLASS_CATEGORY"] == '01 ONE FAMILY DWELLINGS') & 
   (1e5 <df["SALE_PRICE"]) & (df["SALE_PRICE"]<2e6)]

In [17]:
df.shape

(3151, 21)

In [18]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,...,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,...,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,...,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,...,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,...,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,...,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3151 entries, 44 to 23035
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BOROUGH                         3151 non-null   object        
 1   NEIGHBORHOOD                    3151 non-null   object        
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object        
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object        
 4   BLOCK                           3151 non-null   int64         
 5   LOT                             3151 non-null   int64         
 6   EASE-MENT                       0 non-null      float64       
 7   BUILDING_CLASS_AT_PRESENT       3151 non-null   object        
 8   ADDRESS                         3151 non-null   object        
 9   APARTMENT_NUMBER                1 non-null      object        
 10  ZIP_CODE                        3151 non-null   float64       
 11  RE

In [24]:
# Convert SALE_DATE to date time format
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)

In [26]:
cutoff = pd.to_datetime('04-01-2019')
# cutoff = pd.to_datetime('2019-04-01')
df[df['SALE_DATE'] <= cutoff]
train = df[df['SALE_DATE'] <= cutoff]
test  = df[df['SALE_DATE'] > cutoff]

In [27]:
train.describe(exclude='number')

/Users/shahnam/.pyenv/versions/3.7.8/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2548,2548,2548,2548,2548,2548,1,2548,2548,2548
unique,5,7,1,2,13,2538,1,901,11,69
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,125-27 LUCAS STREET,RP.,"4,000",A1,2019-01-31 00:00:00
freq,1226,2400,2548,2515,933,2,1,236,934,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-01 00:00:00


In [28]:
train[["BOROUGH", "NEIGHBORHOOD"]].head(20)

,BOROUGH,NEIGHBORHOOD
44,3,OTHER
61,4,OTHER
78,2,OTHER
108,3,OTHER
111,3,OTHER
120,3,OTHER
121,3,OTHER
132,4,OTHER
137,4,OTHER
141,4,FLUSHING-NORTH


In [29]:
cat_feat = ["NEIGHBORHOOD"]

In [30]:
from sklearn.preprocessing import OneHotEncoder

In [31]:
ohe = OneHotEncoder(sparse=False)
train_trans = ohe.fit_transform(train[cat_feat])
print(type(train_trans))
print(train_trans[:25])

<class 'numpy.ndarray'>
[[0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]]


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2548 entries, 44 to 18433
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BOROUGH                         2548 non-null   object        
 1   NEIGHBORHOOD                    2548 non-null   object        
 2   BUILDING_CLASS_CATEGORY         2548 non-null   object        
 3   TAX_CLASS_AT_PRESENT            2548 non-null   object        
 4   BLOCK                           2548 non-null   int64         
 5   LOT                             2548 non-null   int64         
 6   EASE-MENT                       0 non-null      float64       
 7   BUILDING_CLASS_AT_PRESENT       2548 non-null   object        
 8   ADDRESS                         2548 non-null   object        
 9   APARTMENT_NUMBER                1 non-null      object        
 10  ZIP_CODE                        2548 non-null   float64       
 11  RE

* Total Unit is linearly related to residential and commercial unit columns.
The redundancy will be removed

In [33]:
train[["BLOCK", "LOT", "ZIP_CODE", "RESIDENTIAL_UNITS", "COMMERCIAL_UNITS", "TOTAL_UNITS"]][-10:]

,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS
18414,12588,55,11434.0,1.0,0.0,1.0
18419,8909,38,11421.0,1.0,0.0,1.0
18424,3651,95,10305.0,1.0,0.0,1.0
18426,4309,65,10306.0,2.0,0.0,2.0
18427,2402,50,10314.0,1.0,0.0,1.0
18428,2445,17,10314.0,1.0,0.0,1.0
18430,4007,133,10306.0,1.0,0.0,1.0
18431,7702,304,10309.0,1.0,0.0,1.0
18432,2996,84,10305.0,1.0,0.0,1.0
18433,7066,42,10309.0,1.0,0.0,1.0


In [34]:
train = train.drop(labels=["TOTAL_UNITS", "COMMERCIAL_UNITS"], axis=1)
test = test.drop(labels=["TOTAL_UNITS", "COMMERCIAL_UNITS"], axis=1)

In [35]:
print(train.shape)
test.shape

(2548, 19)


(603, 19)

In [36]:
train.columns[~train.columns.isin(["SALE_PRICE"])]

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET',
       'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_DATE'],
      dtype='object')

In [55]:
train.shape

(2548, 19)

44       550000
61       200000
78       810000
108      125000
111      620000
          ...  
18428    570000
18430    610000
18431    518000
18432    489000
18433    400000
Name: SALE_PRICE, Length: 2548, dtype: int64

In [69]:
print(train.columns[0], type(train.columns))
train.columns[train.columns.isin(["SALE_PRICE"])]

BOROUGH <class 'pandas.core.indexes.base.Index'>


Index(['SALE_PRICE'], dtype='object')

In [76]:
train['APARTMENT_NUMBER'].unique()

array([nan, 'RP.'], dtype=object)

In [78]:
print(train[train.columns[~train.columns.isin(["SALE_PRICE"])]].shape)
print(train[train.columns[~train.columns.isin(["SALE_PRICE"])]].dropna(axis=1).shape)


(2548, 18)
(2548, 16)


* Dropping non-numeric and all columns with any na value

In [79]:
y_train = train['SALE_PRICE']
y_test = test['SALE_PRICE']

# DataFrame[column_index[boolean_index]]
X_train = train[train.columns[~train.columns.isin(["SALE_PRICE"])]].select_dtypes(
    include='number').dropna(how='any',axis=1)

X_test = test[train.columns[~train.columns.isin(["SALE_PRICE"])]].select_dtypes(
    include='number').dropna(how='any', axis=1)

In [80]:
X_train.head()

,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,5495,801,11230.0,1.0,1325.0,1930.0,1
61,7918,72,11427.0,1.0,2001.0,1940.0,1
78,4210,19,10461.0,1.0,2043.0,1925.0,1
108,5212,69,11226.0,1.0,2680.0,1899.0,1
111,7930,121,11203.0,1.0,1872.0,1940.0,1


In [81]:
X_train.shape

(2548, 7)

In [88]:
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(k=4)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

/Users/shahnam/.pyenv/versions/3.7.8/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [6] are constant.
  UserWarning)
/Users/shahnam/.pyenv/versions/3.7.8/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [92]:
# X_train_selected does not have a .columns attribute as it's a numpy array
X_train_selected.shape, type(X_train_selected)

((2548, 4), numpy.ndarray)

In [93]:
col_selected = X_train.columns[selector.get_support()]
col_selected

Index(['LOT', 'RESIDENTIAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT'], dtype='object')

### ole Regression

In [94]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_selected, y_train)
y_train_pred = lr.predict(X_train_selected)
y_test_pred = lr.predict(X_test_selected)

**Score metrics**

In [95]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [96]:
mse_test = mean_squared_error(y_test, y_test_pred)
mse_train = mean_squared_error(y_train, y_train_pred)

rmse_test = np.sqrt(mse_test)
rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)

mae_test = mean_absolute_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f'Mean Squared Errors(Train, Test):, {mse_train:.2e}, {mse_test:.2e}')
print(f'Root Mean Squared Error(Train, Test):, {rmse_train:.2e}, {rmse_test:.2e}')
print(f'Mean Absolute Error(Train, Test):, {mae_train:.2e}, {mae_test:.2e}')
print(f'R^2(Train, Test):, {r2_train:.3f}, {r2_test:.3f}')

Mean Squared Errors(Train, Test):, 6.61e+10, 6.92e+10
Root Mean Squared Error(Train, Test):, 2.57e+05, 2.63e+05
Mean Absolute Error(Train, Test):, 1.91e+05, 1.85e+05
R^2(Train, Test):, 0.224, 0.213


**R^2 greater than zero shows some improvement on linear regression vs. mean baseline**

R2 = 1 - Sigma(y-ypred)/Sigma(y-y_bar) = 1 - unexplained variance of y by the model/total variance of y

## Ridge Regression

In [97]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

def myridge(ridge_inst):
  ridge_inst.fit(X_train_selected, y_train)

  y_train_pred = ridge_inst.predict(X_train_selected)
  y_test_pred = ridge_inst.predict(X_test_selected)

  r2_train = r2_score(y_train, y_train_pred)
  r2_test = r2_score(y_test, y_test_pred)
  return (r2_train, r2_test)


In [98]:
for alpha in [0.01, 0.1, 1, 5, 10]:
  ridge = Ridge(alpha=alpha, normalize=True)
  r2_train, r2_test = myridge(ridge)
  print(f'Ridge R2(Train, Test):, {r2_train:.3f}, {r2_test:.3f}, for alpha: {alpha}')


Ridge R2(Train, Test):, 0.223, 0.212, for alpha: 0.01
Ridge R2(Train, Test):, 0.221, 0.207, for alpha: 0.1
Ridge R2(Train, Test):, 0.164, 0.142, for alpha: 1
Ridge R2(Train, Test):, 0.067, 0.050, for alpha: 5
Ridge R2(Train, Test):, 0.038, 0.023, for alpha: 10


**RidgeCV does not require a loop over alpha as it does check each alpha in a crossfold and decides which alpha is the best**

cv=None would leave one out for validation

In [108]:
alphas = [0.001, 0.01, 0.05, 0.1, 0.5, 1, 10, 100]

from sklearn.linear_model import RidgeCV
ridge = RidgeCV(alphas=alphas, normalize=True, cv=10)
ridge.fit(X_train_selected, y_train)

y_train_pred = ridge.predict(X_train_selected)
y_test_pred = ridge.predict(X_test_selected)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f'RidgeCV R2(Train, Test):, {r2_train:.3f}, {r2_test:.3f}, for alpha: {ridge.alpha_}')


RidgeCV R2(Train, Test):, 0.223, 0.210, for alpha: 0.05
